In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pose_dataset import PoseData

In [2]:
INPUT_SIZE = 33 * 3
HIDDEN_SIZE = 64
NUM_LAYERS = 2 # number of RNNs to stack
NUM_CLASSES = 9 # number of categories

LEARNING_RATE = 0.1

In [3]:
TIME_DIM = 0
BATCH_DIM = 1
COORD_DIM = 2

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cpu


In [ ]:
class PoseScoringModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(PoseScoringModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        h0 = torch.zeros(self.num_layers, X.size(BATCH_DIM), self.hidden_size).to(device)

        out, _ = self.rnn(X, h0)
        # out: batch x time x hidden
        out = out[:, -1, :]
        # out: batch x hidden
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

In [ ]:
model = PoseScoringModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES)

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
NUM_WORKERS = 2

In [ ]:
data = PoseData("PoseData")
data.shuffle()

train_data = data[:len(data) * .9]
test_data = data[len(data) * .9:]

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)
# train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
# test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

In [ ]:
EPOCHS = 10  # Number of epochs for training

for epoch in range(EPOCHS):
    model.train()  # Set the model to training mode
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss_value = loss(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{EPOCHS}], Loss: {loss_value.item():.4f}")

Epoch [1/10], Loss: 2.2096
Epoch [2/10], Loss: 2.1685
Epoch [3/10], Loss: 2.2148
Epoch [4/10], Loss: 2.3354
Epoch [5/10], Loss: 2.1534
Epoch [6/10], Loss: 2.3413
Epoch [7/10], Loss: 2.3165
Epoch [8/10], Loss: 2.3018
Epoch [9/10], Loss: 2.3047
Epoch [10/10], Loss: 2.3223


In [ ]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
eval_loss = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, targets in test_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        eval_loss += loss(outputs, targets).item()

        # Get predictions and calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Evaluation Loss: {eval_loss:.4f}, Accuracy: {accuracy:.2f}%")